In [1]:
import pandas as pd
import numpy as np

In [3]:
walmart_data = pd.read_csv("Walmart Sales.csv")

In [4]:
walmart_data

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Date,Time,Payment,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,1/5/2019,13:08,Ewallet,9.1
1,226-31-3081,A,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3/8/2019,10:29,Cash,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,3/3/2019,13:23,Credit card,7.4
3,123-19-1176,B,Yangon,Member,Male,Health and beauty,58.22,8,1/27/2019,20:33,Ewallet,8.4
4,373-73-7910,C,Yangon,Normal,Male,Sports and travel,86.31,7,2/8/2019,10:37,Ewallet,5.3
...,...,...,...,...,...,...,...,...,...,...,...,...
995,233-67-5758,A,Naypyitaw,Normal,Male,Health and beauty,40.35,1,1/29/2019,13:46,Ewallet,6.2
996,303-96-2227,A,Mandalay,Normal,Female,Home and lifestyle,97.38,10,3/2/2019,17:16,Ewallet,4.4
997,727-02-1313,A,Yangon,Member,Male,Food and beverages,31.84,1,2/9/2019,13:22,Cash,7.7
998,347-56-2442,B,Yangon,Normal,Male,Home and lifestyle,65.82,1,2/22/2019,15:33,Cash,4.1


In [5]:
print(walmart_data.isnull().sum())

Invoice ID       0
Branch           0
City             0
Customer type    0
Gender           0
Product line     0
Unit price       0
Quantity         0
Date             0
Time             0
Payment          0
Rating           0
dtype: int64


In [6]:
# Calculate total sales and revenue
walmart_data['Total Sales'] = walmart_data['Unit price'] * walmart_data['Quantity']

In [8]:
print(walmart_data['Total Sales'])

0      522.83
1       76.40
2      324.31
3      465.76
4      604.17
        ...  
995     40.35
996    973.80
997     31.84
998     65.82
999    618.38
Name: Total Sales, Length: 1000, dtype: float64


In [18]:
# Calculate total sales
walmart_data['Total Sales'] = walmart_data['Unit price'] * walmart_data['Quantity']

# Create 'Revenue' column
walmart_data['Revenue'] = walmart_data['Total Sales']

# Aggregate data at city and branch level
city_branch_performance = walmart_data.groupby(['City', 'Branch']).agg({
    'Total Sales': 'sum',
    'Revenue': 'sum'
})

# Analyze performance
city_branch_performance['Average Sales'] = city_branch_performance['Total Sales'] / walmart_data.groupby(['City', 'Branch'])['Invoice ID'].nunique()
print(city_branch_performance['Average Sales'])
# Visualize data
#city_branch_performance.plot(kind='bar', figsize=(10, 6), title='City and Branch Performance')


City       Branch
Mandalay   A         286.807479
           B         315.389237
           C         313.627579
Naypyitaw  A         327.142182
           B         322.548165
           C         313.395780
Yangon     A         295.151491
           B         311.446991
           C         285.862212
Name: Average Sales, dtype: float64


In [19]:
# Group data by City and Branch, calculate average price
average_price_per_branch = walmart_data.groupby(['City', 'Branch'])['Unit price'].mean()

# Display the result
print("Average price of an item sold at each branch of the city:")
print(average_price_per_branch)


Average price of an item sold at each branch of the city:
City       Branch
Mandalay   A         53.353866
           B         56.133305
           C         57.958316
Naypyitaw  A         54.123182
           B         57.785688
           C         57.941009
Yangon     A         55.639298
           B         56.011062
           C         52.684602
Name: Unit price, dtype: float64


In [40]:
walmart_data['Date'] = pd.to_datetime(walmart_data['Date'])
walmart_data['Month'] = walmart_data['Date'].dt.month
walmart_data['Year'] = walmart_data['Date'].dt.year

monthly_performance = walmart_data.groupby(['Year', 'Month', 'Product line', 'Gender', 'Payment']).agg({
    'Quantity': 'sum',
    'Unit price': 'mean',
    'Total Sales': 'sum'
}).reset_index()
print(monthly_performance)

     Year  Month            Product line  Gender      Payment  Quantity  \
0    2019      1  Electronic accessories  Female         Cash        52   
1    2019      1  Electronic accessories  Female  Credit card        54   
2    2019      1  Electronic accessories  Female      Ewallet        43   
3    2019      1  Electronic accessories    Male         Cash        62   
4    2019      1  Electronic accessories    Male  Credit card        43   
..    ...    ...                     ...     ...          ...       ...   
103  2019      3       Sports and travel  Female  Credit card        52   
104  2019      3       Sports and travel  Female      Ewallet        53   
105  2019      3       Sports and travel    Male         Cash        36   
106  2019      3       Sports and travel    Male  Credit card        60   
107  2019      3       Sports and travel    Male      Ewallet        86   

     Unit price  Total Sales  
0     59.386667      2731.86  
1     53.028571      3045.42  
2     

In [39]:
# Compare sales and revenue for April 2019 with previous months
april_2019_data = monthly_performance[(monthly_performance['Year'] == 2019) & (monthly_performance['Month'] == 4)]
previous_months_data = monthly_performance[(monthly_performance['Year'] == 2019) & (monthly_performance['Month'] < 4)]
#print(april_2019_data)
print(previous_months_data,"\n")
print("Calculate Average Sales for Previous Months\n\n")
# Calculate average sales for each dimension (Product line, Gender, Payment) for previous months
average_sales_previous_months = previous_months_data.groupby(['Product line', 'Gender', 'Payment'])['Total Sales'].mean().reset_index()
print(average_sales_previous_months)

# Merge April 2019 data with average sales for previous months
merged_data = april_2019_data.merge(average_sales_previous_months, on=['Product line', 'Gender', 'Payment'], suffixes=('_April_2019', '_Previous_Months'))
#print(merged_data)

# Identify areas with lower performance in April 2019 compared to previous months
lower_performance_areas = merged_data[merged_data['Total Sales_April_2019'] < merged_data['Total Sales_Previous_Months']]



     Year  Month            Product line  Gender      Payment  Quantity  \
0    2019      1  Electronic accessories  Female         Cash        52   
1    2019      1  Electronic accessories  Female  Credit card        54   
2    2019      1  Electronic accessories  Female      Ewallet        43   
3    2019      1  Electronic accessories    Male         Cash        62   
4    2019      1  Electronic accessories    Male  Credit card        43   
..    ...    ...                     ...     ...          ...       ...   
103  2019      3       Sports and travel  Female  Credit card        52   
104  2019      3       Sports and travel  Female      Ewallet        53   
105  2019      3       Sports and travel    Male         Cash        36   
106  2019      3       Sports and travel    Male  Credit card        60   
107  2019      3       Sports and travel    Male      Ewallet        86   

     Unit price  Total Sales  
0     59.386667      2731.86  
1     53.028571      3045.42  
2     